In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score
from xgboost import XGBRegressor, XGBClassifier
from load import dataset

In [2]:
# TODO
#

In [67]:
# Load
df = dataset(predict_season=2020, predict_week=1, window_size=4, update_data=True, update_seasons=[pd.to_datetime('today').year])

In [68]:
# Clean Up
df[df.columns[df.columns.str.contains('home|Home')]] = df[df.columns[df.columns.str.contains(
    'home|Home')]].groupby('home_team', as_index=False, group_keys=False).apply(lambda x: x.fillna(x.mean()))

df[df.columns[df.columns.str.contains('away|Away')]] = df[df.columns[df.columns.str.contains(
    'away|Away')]].groupby('away_team', as_index=False, group_keys=False).apply(lambda x: x.fillna(x.mean()))

df = df.dropna(axis=1, thresh=int(len(df) * .9))

df = df.fillna(0)

df[df.select_dtypes('bool').columns] = df.select_dtypes('bool').astype('int')

In [69]:
# Model
ct = make_column_transformer(
    (OrdinalEncoder(), ['season_type', 'home_conference', 'away_conference']),
    remainder='passthrough'
)

drop_columns =  ['game_id', 'season', 'venue', 'start_date', 'home_team', 'away_team', 'start_date', 'spread_target']

X_train = df[df.start_date.dt.date < pd.to_datetime('today')].drop(drop_columns, axis=1)  
y_train = df[df.start_date.dt.date < pd.to_datetime('today')][['spread_target']] 

X_pred = df[df.start_date.dt.date >= pd.to_datetime('today')].drop(drop_columns, axis=1) 
# y_pred = y[X.start_date.dt.date >= pd.to_datetime('today')]

spread_model = XGBRegressor(learning_rate=0.1, 
                                colsample_bytree=1.0,
                                gamma=0.5,
                                max_depth=5,
                                min_child_weight=8,
                                n_estimators=100,
                                subsample=1.0)

spread_model.fit(ct.fit_transform(X_train), y_train['spread_target'])
y_spread_pred = np.around(spread_model.predict(ct.fit_transform(X_pred))/.5, decimals=0)*.5

final_columns = ['game_id', 'season', 'week', 'season_type', 'home_team',
                'home_conference', 'away_team', 'away_conference',  'pre_game_spread',
                'predicted_spread']

X_pred = pd.concat([
                    X_pred, 
                    df[df.start_date.dt.date >= pd.to_datetime('today')].drop([col for col in df.columns if col not in drop_columns], axis=1) 
                    ], 
                    axis=1)

predict_df = pd.concat([
        X_pred.reset_index(drop=True), 
        pd.Series(y_spread_pred, name='predicted_spread'),
     ], axis=1)

predict_df = predict_df[final_columns]

predict_df['spread_pick'] = np.where(predict_df['predicted_spread'] <= predict_df['pre_game_spread'], predict_df['home_team'], predict_df['away_team'])
predict_df['straight_pick'] = np.where(predict_df['predicted_spread'] < 0, predict_df['home_team'], predict_df['away_team'])

predict_df

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,spread_pick,straight_pick
0,401234576,2020,1,regular,Navy,American Athletic,BYU,FBS Independents,1.0,-8.0,Navy,Navy


In [70]:
# Pull in games that have occurred within the current week to append to the predictions
filepath = Path('../zillion_picks/picks')

result_dfs = []

for file in filepath.rglob('*.csv'):
    result_df = pd.read_csv(file)

    result_dfs.append(file_df)

result_df = pd.concat(file_dfs)

result_df = result_df[(result_df['week'] == predict_df['week'][0]) & (result_df['season'] == predict_df['season'][0])]

result_df = result_df[~result_df.game_id.isin(predict_df.game_id)]
result_df = result_df.drop(['actual_spread', 'spread_result', 'straight_result', 'start_date'], axis=1)


In [73]:
# Merge Predit and Result Dataframes

predict_df = pd.concat([result_df, predict_df])

In [74]:
# Add spread target in case the game is complete
predict_df = predict_df.merge(df[['game_id', 'spread_target', 'start_date']], on='game_id', how='left')

In [75]:
predict_df

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,spread_pick,straight_pick,spread_target,start_date
0,401238035,2020,1,regular,UAB,Conference USA,Central Arkansas,MISSING,-21.0,-29.0,UAB,UAB,-10.0,2020-09-04 00:00:00+00:00
1,401207101,2020,1,regular,Southern Mississippi,Conference USA,South Alabama,Sun Belt,-12.5,-2.5,South Alabama,Southern Mississippi,11.0,2020-09-04 01:00:00+00:00
2,401235700,2020,1,regular,Army,FBS Independents,Middle Tennessee,Conference USA,-4.0,-16.5,Army,Army,-42.0,2020-09-05 17:30:00+00:00
3,401212484,2020,1,regular,Texas State,Sun Belt,SMU,American Athletic,23.0,29.5,SMU,SMU,7.0,2020-09-05 20:30:00+00:00
4,401234576,2020,1,regular,Navy,American Athletic,BYU,FBS Independents,1.0,-8.0,Navy,Navy,0.0,2020-09-08 00:00:00+00:00


In [76]:
# Convert spread_target games that have not occurred to Null
predict_df['spread_target']  = np.where(pd.to_datetime(predict_df['start_date']).dt.date > pd.to_datetime('today').date(), np.nan, predict_df['spread_target'] )

In [77]:
predict_df

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,spread_pick,straight_pick,spread_target,start_date
0,401238035,2020,1,regular,UAB,Conference USA,Central Arkansas,MISSING,-21.0,-29.0,UAB,UAB,-10.0,2020-09-04 00:00:00+00:00
1,401207101,2020,1,regular,Southern Mississippi,Conference USA,South Alabama,Sun Belt,-12.5,-2.5,South Alabama,Southern Mississippi,11.0,2020-09-04 01:00:00+00:00
2,401235700,2020,1,regular,Army,FBS Independents,Middle Tennessee,Conference USA,-4.0,-16.5,Army,Army,-42.0,2020-09-05 17:30:00+00:00
3,401212484,2020,1,regular,Texas State,Sun Belt,SMU,American Athletic,23.0,29.5,SMU,SMU,7.0,2020-09-05 20:30:00+00:00
4,401234576,2020,1,regular,Navy,American Athletic,BYU,FBS Independents,1.0,-8.0,Navy,Navy,NaN,2020-09-08 00:00:00+00:00


In [78]:
# Spread Result Calculation
won = ((predict_df['predicted_spread'] > predict_df['pre_game_spread']) & (predict_df['spread_target'].fillna(0) > predict_df['pre_game_spread'])) | (
    (predict_df['predicted_spread'] < predict_df['pre_game_spread']) & (predict_df['spread_target'].fillna(0) < predict_df['pre_game_spread']))

push = predict_df['spread_target'].fillna(0) == predict_df['pre_game_spread']

conditions = [won, push]
choices = ['Won', 'Push']

predict_df['spread_result'] = np.select(condlist=conditions, choicelist=choices, default='Lost')
predict_df['spread_result'] = np.where(predict_df['spread_target'].isnull(), np.nan, predict_df['spread_result'])

In [79]:
# Straight Up Result Calculation
result = predict_df['spread_target'].fillna(1) * predict_df['predicted_spread']

predict_df['straight_result'] = np.where(result > 0, 'Won', 'Lost')
predict_df['straight_result'] = np.where(predict_df['spread_target'].isnull(), np.nan, predict_df['straight_result'])

In [80]:
# Rename Column
predict_df = predict_df.rename(columns={'spread_target':'actual_spread'})

In [81]:
predict_df

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,spread_pick,straight_pick,actual_spread,start_date,spread_result,straight_result
0,401238035,2020,1,regular,UAB,Conference USA,Central Arkansas,MISSING,-21.0,-29.0,UAB,UAB,-10.0,2020-09-04 00:00:00+00:00,Lost,Won
1,401207101,2020,1,regular,Southern Mississippi,Conference USA,South Alabama,Sun Belt,-12.5,-2.5,South Alabama,Southern Mississippi,11.0,2020-09-04 01:00:00+00:00,Won,Lost
2,401235700,2020,1,regular,Army,FBS Independents,Middle Tennessee,Conference USA,-4.0,-16.5,Army,Army,-42.0,2020-09-05 17:30:00+00:00,Won,Won
3,401212484,2020,1,regular,Texas State,Sun Belt,SMU,American Athletic,23.0,29.5,SMU,SMU,7.0,2020-09-05 20:30:00+00:00,Lost,Won
4,401234576,2020,1,regular,Navy,American Athletic,BYU,FBS Independents,1.0,-8.0,Navy,Navy,NaN,2020-09-08 00:00:00+00:00,NaN,NaN


In [82]:
# Export
filepath = Path('../zillion_picks/picks')

for key, group in predict_df.groupby(['season', 'week', 'season_type']):
    if group.season_type.unique() == 'postseason':
        group.to_csv(filepath/f'premium/{key[0]}_postseason.csv', index=False)
    else:
        group.to_csv(filepath/f'premium/{key[0]}_{str(key[1]).rjust(2, "0")}.csv', index=False)